In [1]:
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if len(gpus) > 1:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[1], True)
print(tf.__version__)
print(tf.sysconfig.get_build_info()["cuda_version"])
print(tf.sysconfig.get_build_info()["cudnn_version"])

2025-04-01 09:17:22.257702: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 09:17:22.257797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 09:17:22.259584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 09:17:22.271728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.15.0
12.0
8


In [2]:
import numpy as np
import warnings
import resampy
from scipy.io import wavfile
from python_speech_features import mfcc
import tensorflow as tf
import time

# tf.compat.v1.config.optimizer.set_jit(True)

class DeepSpeech():
    def __init__(self,model_path):
        self.graph = tf.Graph()
        with self.graph.as_default():
            with tf.io.gfile.GFile(model_path, "rb") as f:
                graph_def = tf.compat.v1.GraphDef()
                graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name="deepspeech")
            self.logits_ph = self.graph.get_tensor_by_name("deepspeech/logits:0")
            print("self.logits_ph", self.logits_ph)
            self.input_node_ph = self.graph.get_tensor_by_name("deepspeech/input_node:0")
            print("self.input_node_ph", self.input_node_ph)
            self.input_lengths_ph = self.graph.get_tensor_by_name("deepspeech/input_lengths:0")
            print("self.input_lengths_ph", self.input_lengths_ph)
            config = tf.compat.v1.ConfigProto()
            config.gpu_options.allow_growth = True
            config.gpu_options.per_process_gpu_memory_fraction = 0.5
            config.graph_options.optimizer_options.global_jit_level = tf.compat.v1.OptimizerOptions.ON_1
            config.log_device_placement = True
            self.session = tf.compat.v1.Session(config=config)
        self.target_sample_rate = 16000

    def _prepare_deepspeech_net(self,deepspeech_pb_path):
        tf.compat.v1.disable_eager_execution()  # Bật chế độ tương thích TF1.x
        with tf.io.gfile.GFile(deepspeech_pb_path, "rb") as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())
        graph = tf.compat.v1.get_default_graph()
        tf.import_graph_def(graph_def, name="deepspeech")
        logits_ph = graph.get_tensor_by_name("deepspeech/logits:0")
        input_node_ph = graph.get_tensor_by_name("deepspeech/input_node:0")
        input_lengths_ph = graph.get_tensor_by_name("deepspeech/input_lengths:0")

        return graph, logits_ph, input_node_ph, input_lengths_ph

    def conv_audio_to_deepspeech_input_vector(self,audio,
                                              sample_rate,
                                              num_cepstrum,
                                              num_context):
        # Get mfcc coefficients:
        features = mfcc(
            signal=audio,
            samplerate=sample_rate,
            numcep=num_cepstrum)

        # We only keep every second feature (BiRNN stride = 2):
        features = features[::2]

        # One stride per time step in the input:
        num_strides = len(features)

        # Add empty initial and final contexts:
        empty_context = np.zeros((num_context, num_cepstrum), dtype=features.dtype)
        features = np.concatenate((empty_context, features, empty_context))

        # Create a view into the array with overlapping strides of size
        # numcontext (past) + 1 (present) + numcontext (future):
        window_size = 2 * num_context + 1
        train_inputs = np.lib.stride_tricks.as_strided(
            features,
            shape=(num_strides, window_size, num_cepstrum),
            strides=(features.strides[0],
                     features.strides[0], features.strides[1]),
            writeable=False)

        # Flatten the second and third dimensions:
        train_inputs = np.reshape(train_inputs, [num_strides, -1])

        train_inputs = np.copy(train_inputs)
        train_inputs = (train_inputs - np.mean(train_inputs)) / \
                       np.std(train_inputs)

        return train_inputs

    def compute_audio_feature(self, audio_path):
        start_time = time.time()
        audio_sample_rate, audio = wavfile.read(audio_path)
        print(audio_sample_rate)
        if audio.ndim != 1:
            warnings.warn(
                "Audio has multiple channels, the first channel is used")
            audio = audio[:, 0]
        if audio_sample_rate != self.target_sample_rate:
            resampled_audio = resampy.resample(
                x=audio.astype(float),
                sr_orig=audio_sample_rate,
                sr_new=self.target_sample_rate)
        else:
            # resampled_audio = audio.astype(np.float)
            resampled_audio = audio.astype(float)
        end_time = time.time()
        print(f"Time taken to resample audio: {end_time - start_time} seconds")

        start_time = time.time()
        input_vector = self.conv_audio_to_deepspeech_input_vector(
            audio=resampled_audio.astype(np.int16),
            sample_rate=self.target_sample_rate,
            num_cepstrum=26,
            num_context=9)
        end_time = time.time()
        print("input_vector", input_vector.shape)
        print(f"Time taken to convert audio to input vector: {end_time - start_time} seconds")
        start_time = time.time()
        
        network_output = self.session.run(
                self.logits_ph,
                feed_dict={
                    self.input_node_ph: input_vector[np.newaxis, ...],
                    self.input_lengths_ph: [input_vector.shape[0]]
                })
        print("network_output", network_output.shape)
        end_time = time.time()
        print(f"Time taken to run network: {end_time - start_time} seconds")
        ds_features = network_output[::2,0,:]
        print("ds_features", ds_features.shape)
        return ds_features

DSModel = DeepSpeech('./asserts/output_graph.pb')

# input_vector.shape = (410, 494) = (num_strides(=410), window_size(=2*num_context+1=2*8+1=19) * num_cepstrum(=26) )
# network_output.shape = (410, 1, 29) = (num_strides(=410), batch_size(=1), num_classes(=29))

self.logits_ph Tensor("deepspeech/logits:0", shape=(None, None, 29), dtype=float32)
self.input_node_ph Tensor("deepspeech/input_node:0", shape=(None, None, 494), dtype=float32)
self.input_lengths_ph Tensor("deepspeech/input_lengths:0", shape=(None,), dtype=int32)
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:af:00.0, compute capability: 8.9



2025-04-01 09:17:40.859826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12107 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:af:00.0, compute capability: 8.9


In [5]:
ds_feature = DSModel.compute_audio_feature('output_audio.wav')
print("ds_feature", ds_feature.shape)

24000
Time taken to resample audio: 0.14072537422180176 seconds
input_vector (410, 494)
Time taken to convert audio to input vector: 0.022240877151489258 seconds
network_output (410, 1, 29)
Time taken to run network: 0.20188570022583008 seconds
ds_features (205, 29)
ds_feature (205, 29)


In [ ]:
import numpy as np
import warnings
import resampy
from scipy.io import wavfile
from python_speech_features import mfcc
import tensorflow as tf
import time

class DeepSpeech():
    def __init__(self, model_path):
        # TensorFlow 2.x không cần phải sử dụng tf.Graph() như TensorFlow 1.x
        # Hãy tải mô hình trực tiếp bằng cách sử dụng tf.saved_model.load()
        self.model = tf.saved_model.load(model_path)
        self.target_sample_rate = 16000
        
        # Lấy các tensor từ mô hình
        self.input_node_ph = self.model.signatures['serving_default'].inputs[0]  # Xác định tên của input tensor
        self.logits_ph = self.model.signatures['serving_default'].outputs[0]  # Xác định tên của output tensor
        print(f"Input node: {self.input_node_ph}, Logits node: {self.logits_ph}")
    
    def conv_audio_to_deepspeech_input_vector(self, audio, sample_rate, num_cepstrum, num_context):
        # Extract mfcc features
        features = mfcc(signal=audio, samplerate=sample_rate, numcep=num_cepstrum)
        features = features[::2]  # Keep every second feature
        num_strides = len(features)

        # Add empty initial and final contexts
        empty_context = np.zeros((num_context, num_cepstrum), dtype=features.dtype)
        features = np.concatenate((empty_context, features, empty_context))

        window_size = 2 * num_context + 1
        train_inputs = np.lib.stride_tricks.as_strided(features, shape=(num_strides, window_size, num_cepstrum), 
                                                       strides=(features.strides[0], features.strides[0], features.strides[1]), 
                                                       writeable=False)
        train_inputs = np.reshape(train_inputs, [num_strides, -1])
        train_inputs = np.copy(train_inputs)
        train_inputs = (train_inputs - np.mean(train_inputs)) / np.std(train_inputs)

        return train_inputs

    def compute_audio_feature(self, audio_path):
        start_time = time.time()
        audio_sample_rate, audio = wavfile.read(audio_path)
        print(audio_sample_rate)
        if audio.ndim != 1:
            warnings.warn("Audio has multiple channels, the first channel is used")
            audio = audio[:, 0]
        if audio_sample_rate != self.target_sample_rate:
            resampled_audio = resampy.resample(x=audio.astype(float), sr_orig=audio_sample_rate, sr_new=self.target_sample_rate)
        else:
            resampled_audio = audio.astype(float)
        end_time = time.time()
        print(f"Time taken to resample audio: {end_time - start_time} seconds")

        start_time = time.time()
        input_vector = self.conv_audio_to_deepspeech_input_vector(audio=resampled_audio.astype(np.int16), 
                                                                 sample_rate=self.target_sample_rate, 
                                                                 num_cepstrum=26, 
                                                                 num_context=9)
        end_time = time.time()
        print(f"Time taken to convert audio to input vector: {end_time - start_time} seconds")

        start_time = time.time()
        
        # TensorFlow 2.x không cần phải dùng session, chỉ cần gọi mô hình với input
        input_tensor = tf.convert_to_tensor(input_vector[np.newaxis, ...], dtype=tf.float32)
        logits = self.model(input_tensor)
        
        print(f"Logits shape: {logits.shape}")
        end_time = time.time()
        print(f"Time taken to run network: {end_time - start_time} seconds")
        
        ds_features = logits[::2, 0, :]
        print(f"ds_features shape: {ds_features.shape}")
        
        return ds_features

# Đường dẫn tới mô hình .pb đã được lưu
DSModel = DeepSpeech('./asserts/output_graph.pb')

# Lưu ý rằng chúng ta cần chuyển mô hình của bạn sang định dạng TensorFlow SavedModel
# Nếu mô hình của bạn chưa được lưu dưới định dạng này, bạn có thể chuyển đổi bằng cách sử dụng:
# model.save('saved_model/my_model') trước khi chạy.


In [ ]:
import tensorflow as tf

# Tải mô hình .pb đã lưu trước đó
def load_pb_model(pb_path):
    # Tải graph từ file .pb
    with tf.io.gfile.GFile(pb_path, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    
    # Tạo một graph mới và import graph_def vào graph đó
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")
    
    return graph

# Hàm để kiểm tra các tên tensor trong mô hình
def check_tensor_names(pb_path):
    graph = load_pb_model(pb_path)

    # Duyệt qua tất cả các node và in ra tên tensor
    for op in graph.get_operations():
        print(f"Operation: {op.name}")
        for output in op.outputs:
            print(f"    Tensor name: {output.name}")

# kiểm tra tên tensor trong mô hình
pb_model_path = './asserts/output_graph.pb'
check_tensor_names(pb_model_path)

Operation: input_node
    Tensor name: input_node:0
Operation: input_lengths
    Tensor name: input_lengths:0
Operation: ToInt64
    Tensor name: ToInt64:0
Operation: Shape
    Tensor name: Shape:0
Operation: transpose/perm
    Tensor name: transpose/perm:0
Operation: transpose
    Tensor name: transpose:0
Operation: Reshape/shape
    Tensor name: Reshape/shape:0
Operation: Reshape
    Tensor name: Reshape:0
Operation: b1
    Tensor name: b1:0
Operation: b1/read
    Tensor name: b1/read:0
Operation: h1
    Tensor name: h1:0
Operation: h1/read
    Tensor name: h1/read:0
Operation: MatMul
    Tensor name: MatMul:0
Operation: Add
    Tensor name: Add:0
Operation: Relu
    Tensor name: Relu:0
Operation: Minimum/y
    Tensor name: Minimum/y:0
Operation: Minimum
    Tensor name: Minimum:0
Operation: b2
    Tensor name: b2:0
Operation: b2/read
    Tensor name: b2/read:0
Operation: h2
    Tensor name: h2:0
Operation: h2/read
    Tensor name: h2/read:0
Operation: MatMul_1
    Tensor name: MatMu

In [ ]:
# Tạo lớp mô hình có thể theo dõi (Trackable)
class DeepSpeechModel(tf.Module):
    def __init__(self, graph):
        super().__init__()
        self.graph = graph
        self.input_node = graph.get_tensor_by_name("input_node:0")  # Thay đúng tên tensor
        self.logits_node = graph.get_tensor_by_name("logits:0")  # Thay đúng tên tensor
        self.input_lengths_node = graph.get_tensor_by_name("input_lengths:0")  # Thay đúng tên tensor

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 494], dtype=tf.float32)])  # Điều chỉnh shape cho phù hợp
    def __call__(self, input_vector):
        # Mô phỏng quá trình chạy mô hình
        with tf.compat.v1.Session(graph=self.graph) as session:
            network_output = session.run(
                self.logits_node,
                feed_dict={self.input_node: input_vector[np.newaxis, ...],
                           self.input_lengths_node: [input_vector.shape[0]]}
            )
        return network_output

# Chuyển đổi mô hình .pb thành SavedModel
def convert_pb_to_savedmodel(pb_path, saved_model_path):
    graph = load_pb_model(pb_path)
    model = DeepSpeechModel(graph)

    # Lưu mô hình dưới dạng SavedModel
    tf.saved_model.save(model, saved_model_path)
    print(f"Model saved to {saved_model_path}")

# Chuyển đổi mô hình .pb thành SavedModel
pb_model_path = './asserts/output_graph.pb'
saved_model_path = './saved_model/deepspeech_model'
convert_pb_to_savedmodel(pb_model_path, saved_model_path)

In [1]:
import numpy as np
import cv2
import torch
import random

In [2]:
def compute_crop_radius(video_size,landmark_data_clip,random_scale = None):
    '''
    judge if crop face and compute crop radius
    '''
    video_w, video_h = video_size[0], video_size[1]
    landmark_max_clip = np.max(landmark_data_clip, axis=1)
    if random_scale is None:
        random_scale = random.random() / 10 + 1.05
    else:
        random_scale = random_scale

    radius_h = (landmark_max_clip[:,1] - landmark_data_clip[:,29, 1]) * random_scale
    radius_w = (landmark_data_clip[:,54, 0] - landmark_data_clip[:,48, 0]) * random_scale

    radius_clip = np.max(np.stack([radius_h, radius_w],1),1) // 2
    radius_max = np.max(radius_clip)
    # radius_max = (np.int(radius_max/4) + 1 ) * 4

    radius_max = (int(radius_max/4) + 1 ) * 4
    
    radius_max_1_4 = radius_max//4
    clip_min_h = landmark_data_clip[:, 29, 1] - radius_max
    clip_max_h = landmark_data_clip[:, 29, 1] + radius_max * 2  + radius_max_1_4
    clip_min_w = landmark_data_clip[:, 33, 0] - radius_max - radius_max_1_4
    clip_max_w = landmark_data_clip[:, 33, 0] + radius_max + radius_max_1_4
    if min(clip_min_h.tolist() + clip_min_w.tolist()) < 0:
        print("1")
        return False,radius_max
    elif max(clip_max_h.tolist()) > video_h:
        print("2")
        return False,radius_max
    elif max(clip_max_w.tolist()) > video_w:
        print("3")
        return False,radius_max
    elif max(radius_clip) > min(radius_clip) * 1.5:
        print("4")
        return False, radius_max
    else:
        return True,radius_max

In [3]:
def batch_compute_crop_radius(video_size, landmark_data_clip, random_scale=1.05):
    '''
    Batch processing version of crop radius calculation
    '''
    video_w, video_h = video_size[0], video_size[1]
    
    # Calculate max landmarks for each frame in each window
    landmark_max_clip = np.max(landmark_data_clip, axis=2)  # Shape: (batch, window_size, 2)
    
    # Calculate radius components
    radius_h = (landmark_max_clip[:, :, 1] - landmark_data_clip[:, :, 29, 1]) * random_scale
    radius_w = (landmark_data_clip[:, :, 54, 0] - landmark_data_clip[:, :, 48, 0]) * random_scale
    
    # Stack and find max radius for each frame
    radius_clip = np.max(np.stack([radius_h, radius_w], axis=2), axis=2) // 2  # Shape: (batch, window_size)
    
    # Find max radius for each window
    radius_max = np.max(radius_clip, axis=1)  # Shape: (batch,)
    radius_max = (radius_max // 4 + 1) * 4  # Ensure divisible by 4
    
    radius_max_1_4 = radius_max // 4
    
    # Calculate crop boundaries
    clip_min_h = landmark_data_clip[:, :, 29, 1] - radius_max[:, np.newaxis]
    clip_max_h = landmark_data_clip[:, :, 29, 1] + 2 * radius_max[:, np.newaxis] + radius_max_1_4[:, np.newaxis]
    clip_min_w = landmark_data_clip[:, :, 33, 0] - radius_max[:, np.newaxis] - radius_max_1_4[:, np.newaxis]
    clip_max_w = landmark_data_clip[:, :, 33, 0] + radius_max[:, np.newaxis] + radius_max_1_4[:, np.newaxis]
    
    # Check validity for each window
    valid = (
        (np.all(clip_min_h >= 0, axis=1)) &
        (np.all(clip_max_h <= video_h, axis=1)) &
        (np.all(clip_min_w >= 0, axis=1)) &
        (np.all(clip_max_w <= video_w, axis=1)) &
        (np.max(radius_clip, axis=1) <= 1.5 * np.min(radius_clip, axis=1))
    )
    
    return valid, radius_max

In [37]:
# Giả định đầu vào
frame_h, frame_w = 256, 256
video_size = (frame_w, frame_h)
resize_w, resize_h = 96, 96

# Tạo dữ liệu giả cho landmark: (10 frames, 68 landmarks, 2)
N = 10
landmarks = np.random.randint(50, 200, size=(N, 68, 2)).astype(np.int32)
video_frames = [np.random.randint(0, 255, (frame_h, frame_w, 3), dtype=np.uint8) for _ in range(N)]

# Tạo batch landmarks: (1 batch, 5 frames, 68 landmarks, 2)
clip = landmarks[2:7]
batch_clip = clip[np.newaxis, ...]


# -------- GỌI 2 PHIÊN BẢN HÀM compute_crop_radius --------
# Gốc
flag_ref, radius_ref = compute_crop_radius(video_size, clip, random_scale=1.05)
print("Original compute_crop_radius:", flag_ref, radius_ref)

# Batch
flag_batch, radius_batch = batch_compute_crop_radius(video_size, batch_clip, random_scale=1.05)
print("Batch compute_crop_radius:   ", flag_batch[0], radius_batch[0])

assert flag_ref == flag_batch[0], "Flag mismatch"
assert abs(radius_ref - radius_batch[0]) <= 4, f"Radius mismatch: {radius_ref} vs {radius_batch[0]}"

1
Original compute_crop_radius: False 72
Batch compute_crop_radius:    False 72.0


In [59]:
def crop_and_resize_face(img, landmarks, crop_radius, resize_w, resize_h):
    crop_radius_1_4 = crop_radius // 4
    cropped = img[
        landmarks[29, 1] - crop_radius:landmarks[29, 1] + 2 * crop_radius + crop_radius_1_4,
        landmarks[33, 0] - crop_radius - crop_radius_1_4:landmarks[33, 0] + crop_radius + crop_radius_1_4,
        :
    ]
    original_size = (cropped.shape[1], cropped.shape[0])
    return cv2.resize(cropped, (resize_w, resize_h)), original_size

In [60]:
def batch_crop_and_resize(frames, landmarks, radius_list, resize_w, resize_h):
    batch_output = []
    original_sizes = []
    
    for frame, landmark, radius in zip(frames, landmarks, radius_list):
        try:
            # Ensure radius is integer
            radius = int(round(radius))
            crop_radius_1_4 = radius // 4
            
            # Calculate coordinates and ensure they are integers
            y_start = int(round(landmark[29, 1] - radius))
            y_end = int(round(landmark[29, 1] + 2 * radius + crop_radius_1_4))
            x_start = int(round(landmark[33, 0] - radius - crop_radius_1_4))
            x_end = int(round(landmark[33, 0] + radius + crop_radius_1_4))
            
            # Validate coordinates
            h, w = frame.shape[:2]
            y_start = max(0, y_start)
            y_end = min(h, y_end)
            x_start = max(0, x_start)
            x_end = min(w, x_end)
            
            # Perform cropping
            cropped = frame[y_start:y_end, x_start:x_end, :]
            original_size = (cropped.shape[1], cropped.shape[0])
            
            if cropped.size > 0:
                resized = cv2.resize(cropped, (resize_w, resize_h))
                batch_output.append(resized)
                original_sizes.append(original_size)
            else:
                raise ValueError("Invalid crop region")
                
        except Exception as e:
            print(f"Crop error: {e}, using fallback")
            fallback = np.zeros((resize_h, resize_w, 3), dtype=np.uint8)
            batch_output.append(fallback)
            original_sizes.append((resize_w, resize_h))
    
    return np.array(batch_output), original_sizes

In [73]:
for idx in range(0,1):
    print(idx)

0


In [71]:
# Giả định đầu vào
frame_h, frame_w = 256, 256
video_size = (frame_w, frame_h)
resize_w, resize_h = 96, 96

# Tạo dữ liệu giả cho landmark: (10 frames, 68 landmarks, 2)
N = 10
landmarks = np.random.randint(50, 200, size=(N, 68, 2)).astype(np.int32)
video_frames = [np.random.randint(0, 255, (frame_h, frame_w, 3), dtype=np.uint8) for _ in range(N)]

# -------- GỌI 2 PHIÊN BẢN HÀM crop_and_resize --------
frame_idx = 3
frame = video_frames[frame_idx]
lm = landmarks[frame_idx]

crop1, size1 = crop_and_resize_face(frame, lm, radius_ref, resize_w, resize_h)
crop2_batch, size2_list = batch_crop_and_resize([frame], [lm], [radius_batch[0]], resize_w, resize_h)
crop2 = crop2_batch[0]
size2 = size2_list[0]

# So sánh size
print("Original size:", size1)
print("Batch size:", size2)
assert size1 == size2, f"Original size mismatch: {size1} vs {size2}"

# So sánh nội dung ảnh
print("Cropped image shape:", crop1.shape)
print("Batch cropped image shape:", crop2.shape)
diff = np.abs(crop1.astype(np.int32) - crop2.astype(np.int32))
max_diff = np.max(diff)
print("Max pixel diff:", max_diff)

assert max_diff < 10, "Cropped image content mismatch"

print("✅ Hai phiên bản hoạt động tương đương logic.")


Original size: (180, 234)
Batch size: (180, 234)
Cropped image shape: (96, 96, 3)
Batch cropped image shape: (96, 96, 3)
Max pixel diff: 0
✅ Hai phiên bản hoạt động tương đương logic.
